In [3]:
import pandas as pd
import numpy as np
from itertools import product


def allowed_pairs_for_case(action: str, orig_bit: int):
    """
    action: 'D' or 'C'
    orig_bit: 1 (=G) or 0 (=B) from the 4-bit base norm
    Returns two allowed (mean, nice) pairs as tuples of ints (1=G, 0=B).
    """
    if action == 'D':
        if orig_bit == 1:   # defecting is good
            return [(1,1), (1,0)]  # (G,G) or (G,B)
        else:               # defecting is bad
            return [(0,0), (0,1)]  # (B,B) or (B,G)
    else:  # action == 'C'
        if orig_bit == 1:   # cooperating is good
            return [(1,1), (0,1)]  # (G,G) or (B,G)
        else:               # cooperating is bad
            return [(0,0), (1,0)]  # (B,B) or (G,B)

def compute_leniency_L(bits8_str: str) -> float:
    """XNOR-based leniency over the 4 (mean,nice) pairs."""
    b = [int(x) for x in bits8_str]
    pairs = [(b[i], b[i+1]) for i in range(0, 8, 2)]
    xnor = [1 if m == n else 0 for (m, n) in pairs]
    return sum(xnor) / 4.0

def expand_base_norm_new_rules(base4: str, norm_name: str):
    """
    base4: 4-bit string in (DB, DG, CB, CG) order, e.g. '0011' for IS.
    Returns a DataFrame with all 16 variants under the new rules.
    """
    if len(base4) != 4 or any(c not in '01' for c in base4):
        raise ValueError("base4 must be a 4-bit string like '0011'.")

    # Map the four positions to (action, orig_bit)
    actions = ['D', 'D', 'C', 'C']
    orig = [int(c) for c in base4]  # [DB, DG, CB, CG]

    # For each of the 4 cases, get the 2 allowed (mean,nice) pairs
    choices = [allowed_pairs_for_case(actions[i], orig[i]) for i in range(4)]
    # Cartesian product → 2^4 = 16 variants (one choice per case)
    variants = []
    for idx, pick in enumerate(product(*choices), start=1):
        # pick is a tuple of 4 pairs in order: (DB_pair, DG_pair, CB_pair, CG_pair)
        # Flatten to 8-bit with (mean,nice) order in each case
        bits8 = [pick[0][0], pick[0][1],
                 pick[1][0], pick[1][1],
                 pick[2][0], pick[2][1],
                 pick[3][0], pick[3][1]]
        bits8_str = ''.join(str(x) for x in bits8)

        # Flag: does any original B (0) expand to something other than (B,B)?
        # (Useful if you want to track "expanded_original_B" under new semantics)
        expanded_orig_B = any(
            (orig[i] == 0 and (pick[i][0], pick[i][1]) != (0, 0))
            for i in range(4)
        )

        variants.append({
            "norm": norm_name,
            "variant_id": f"{norm_name}_v{idx}",
            "4bit_orig": base4,
            "8bit_vector": bits8_str,
            "DB_mean": bits8[0], "DB_nice": bits8[1],
            "DG_mean": bits8[2], "DG_nice": bits8[3],
            "CB_mean": bits8[4], "CB_nice": bits8[5],
            "CG_mean": bits8[6], "CG_nice": bits8[7],
            "Emotion_Leniency": compute_leniency_L(bits8_str)
        })

    cols = ["norm","variant_id","4bit_orig","8bit_vector",
            "DB_mean","DB_nice","DG_mean","DG_nice","CB_mean","CB_nice","CG_mean","CG_nice",
            "Emotion_Leniency"]
    return pd.DataFrame(variants)[cols]


In [6]:
stern_judging = expand_base_norm_new_rules("1001", "Stern_Judging")
image_scoring = expand_base_norm_new_rules("0011", "Image_Scoring")
simple_standing = expand_base_norm_new_rules("1011", "Simple_Standing")
shunning = expand_base_norm_new_rules("0001", "Shunning")

In [10]:
all_dfs = [stern_judging, image_scoring, simple_standing, shunning]
total = pd.concat(all_dfs)
total

,norm,variant_id,4bit_orig,8bit_vector,DB_mean,DB_nice,DG_mean,DG_nice,CB_mean,CB_nice,CG_mean,CG_nice,Emotion_Leniency
0,Stern_Judging,Stern_Judging_v1,1001,11000011,1,1,0,0,0,0,1,1,1.00
1,Stern_Judging,Stern_Judging_v2,1001,11000001,1,1,0,0,0,0,0,1,0.75
2,Stern_Judging,Stern_Judging_v3,1001,11001011,1,1,0,0,1,0,1,1,0.75
3,Stern_Judging,Stern_Judging_v4,1001,11001001,1,1,0,0,1,0,0,1,0.50
4,Stern_Judging,Stern_Judging_v5,1001,11010011,1,1,0,1,0,0,1,1,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,Shunning,Shunning_v12,0001,01001001,0,1,0,0,1,0,0,1,0.25
12,Shunning,Shunning_v13,0001,01010011,0,1,0,1,0,0,1,1,0.50
13,Shunning,Shunning_v14,0001,01010001,0,1,0,1,0,0,0,1,0.25
14,Shunning,Shunning_v15,0001,01011011,0,1,0,1,1,0,1,1,0.25


In [12]:
total.to_csv("new_norms.csv", index=False)